In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import cv2
import os
import random
import csv
from PIL import Image
import shutil
from tqdm import tqdm
import torch

from utils.utils0 import tensor_affine_transform, transform_to_displacement_field
from utils.utils1 import transform_points_DVF, ModelParams
from utils.SuperPoint import SuperPointFrontend, PointTracker
from utils.datagen import datagen

nn_thresh = 0.7
superpoint = SuperPointFrontend('utils/superpoint_v1.pth', nms_dist=4,
                          conf_thresh=0.015, nn_thresh=nn_thresh, cuda=True)
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

Device: cuda


In [2]:
model_params = ModelParams(sup=1, dataset=1, image=1, heatmaps=0, 
                           loss_image=1, num_epochs=10, learning_rate=1e-4)
model_params.print_explanation()

Model name:  dataset1_sup1_image1_heatmaps0_loss_image1
Model code:  11101_0.0001_0_10_1
Model params:  {'dataset': 1, 'sup': 1, 'image': 1, 'heatmaps': 0, 'loss_image_case': 1, 'loss_image': NCC(), 'loss_affine': <utils.utils1.loss_affine object at 0x7fc608c1d670>, 'learning_rate': 0.0001, 'decay_rate': 0.96, 'start_epoch': 0, 'num_epochs': 10, 'batch_size': 1, 'model_name': 'dataset1_sup1_image1_heatmaps0_loss_image1'}

Model name:  dataset1_sup1_image1_heatmaps0_loss_image1
Model code:  11101_0.0001_0_10_1
Dataset used:  Synthetic eye easy
Supervised or unsupervised model:  Supervised
Image type:  Image used
Heatmaps used:  Heatmaps not used
Loss function case:  1
Loss function for image:  NCC()
Loss function for affine:  <utils.utils1.loss_affine object at 0x7fc608c1d670>
Learning rate:  0.0001
Decay rate:  0.96
Start epoch:  0
Number of epochs:  10
Batch size:  1




In [3]:
train_dataset, train_df, train_path = datagen(model_params.dataset, True, model_params.sup)
test_dataset, test_df, test_path = datagen(model_params.dataset, False, model_params.sup)

In [14]:
train_df.head()

,source,target,M00,M01,M02,M10,M11,M12,image_path,keypoints
0,Dataset/synth_eye_easy_train/img_0_original.png,Dataset/synth_eye_easy_train/img_0_transformed...,1.0,0.0,-0.004054,0.0,1.0,-0.181688,Dataset/Dataset-processed/15-12-2559/2011248/L...,Dataset/synth_eye_easy_train/img_0_keypoints.csv
1,Dataset/synth_eye_easy_train/img_1_original.png,Dataset/synth_eye_easy_train/img_1_transformed...,1.0,0.0,-0.083248,0.0,1.0,-0.114122,Dataset/Dataset-processed/15-12-2559/2011248/L...,Dataset/synth_eye_easy_train/img_1_keypoints.csv
2,Dataset/synth_eye_easy_train/img_2_original.png,Dataset/synth_eye_easy_train/img_2_transformed...,1.0,0.0,-0.104685,0.0,1.0,0.129880,Dataset/Dataset-processed/15-12-2559/2011248/R...,Dataset/synth_eye_easy_train/img_2_keypoints.csv
3,Dataset/synth_eye_easy_train/img_3_original.png,Dataset/synth_eye_easy_train/img_3_transformed...,1.0,0.0,0.073952,0.0,1.0,0.179092,Dataset/Dataset-processed/15-12-2559/2011248/R...,Dataset/synth_eye_easy_train/img_3_keypoints.csv
4,Dataset/synth_eye_easy_train/img_4_original.png,Dataset/synth_eye_easy_train/img_4_transformed...,1.0,0.0,0.126152,0.0,1.0,-0.152408,Dataset/Dataset-processed/15-12-2559/2011248/R...,Dataset/synth_eye_easy_train/img_4_keypoints.csv


In [9]:
# add one column to the dataframe to store the path to keypoints file
train_df['keypoints'] = train_df['source'].apply(lambda x: x.replace('_original.png', '_keypoints.csv'))
test_df['keypoints'] = test_df['source'].apply(lambda x: x.replace('_original.png', '_keypoints.csv'))

train_df.head()

,source,target,M00,M01,M02,M10,M11,M12,image_path,keypoints
0,Dataset/synth_eye_easy_train/img_0_original.png,Dataset/synth_eye_easy_train/img_0_transformed...,1.0,0.0,-0.004054,0.0,1.0,-0.181688,Dataset/Dataset-processed/15-12-2559/2011248/L...,Dataset/synth_eye_easy_train/img_0_keypoints.csv
1,Dataset/synth_eye_easy_train/img_1_original.png,Dataset/synth_eye_easy_train/img_1_transformed...,1.0,0.0,-0.083248,0.0,1.0,-0.114122,Dataset/Dataset-processed/15-12-2559/2011248/L...,Dataset/synth_eye_easy_train/img_1_keypoints.csv
2,Dataset/synth_eye_easy_train/img_2_original.png,Dataset/synth_eye_easy_train/img_2_transformed...,1.0,0.0,-0.104685,0.0,1.0,0.129880,Dataset/Dataset-processed/15-12-2559/2011248/R...,Dataset/synth_eye_easy_train/img_2_keypoints.csv
3,Dataset/synth_eye_easy_train/img_3_original.png,Dataset/synth_eye_easy_train/img_3_transformed...,1.0,0.0,0.073952,0.0,1.0,0.179092,Dataset/Dataset-processed/15-12-2559/2011248/R...,Dataset/synth_eye_easy_train/img_3_keypoints.csv
4,Dataset/synth_eye_easy_train/img_4_original.png,Dataset/synth_eye_easy_train/img_4_transformed...,1.0,0.0,0.126152,0.0,1.0,-0.152408,Dataset/Dataset-processed/15-12-2559/2011248/R...,Dataset/synth_eye_easy_train/img_4_keypoints.csv


In [ ]:
# save the dataframe to a csv file
train_df.to_csv('Dataset/synth_eye_easy_train.csv', index=False)
test_df.to_csv('Dataset/synth_eye_easy_test.csv', index=False)

In [12]:
train_bar = tqdm(train_dataset, total=len(train_dataset), desc='Train')
for i, data in enumerate(train_bar):

    # Get images and affine parameters
    if model_params.sup:
        source_image, target_image, affine_params_true = data
        affine_params_true = torch.tensor(affine_params_true)
    else:
        source_image, target_image = data
        affine_params_true = None
    source_image = source_image.to(device)
    target_image = target_image.to(device)

    points1, desc1, heatmap1 = superpoint(source_image[0, 0, :, :].cpu().numpy())
    points2, desc2, heatmap2 = superpoint(target_image[0, 0, :, :].cpu().numpy())

    tracker = PointTracker(5, nn_thresh=0.7)
    try:
        matches = tracker.nn_match_two_way(desc1, desc2, nn_thresh=nn_thresh)
    except:
        print('No matches found')
        # TODO: find a better way to do this
        pass

    matches1 = np.array(points1[:2, matches[0, :].astype(int)])
    matches2 = np.array(points2[:2, matches[1, :].astype(int)])
    matches1_2 = transform_points_DVF(torch.tensor(matches1), 
                        affine_params_true, target_image)
    
    # fig, ax = plt.subplots(1, 2)
    # ax[0].imshow(source_image[0, 0, :, :].cpu().detach().numpy())
    # ax[0].plot(matches1[0, :], matches1[1, :], 'r.')
    # ax[1].imshow(target_image[0, 0, :, :].cpu().detach().numpy())
    # ax[1].plot(matches2[0, :], matches2[1, :], 'g.')
    # ax[1].plot(matches1_2[0, :], matches1_2[1, :], 'r.')
    # plt.show()

    # create a dataframe with the matches
    df = pd.DataFrame({'x1': matches1[0, :], 'y1': matches1[1, :],
                       'x2': matches2[0, :], 'y2': matches2[1, :],
                       'x2_': matches1_2[0, :], 'y2_': matches1_2[1, :]})
    save_name = train_df['keypoints'].iloc[i]
    df.to_csv(save_name, index=False)

Train:   0%|          | 0/200 [00:00<?, ?it/s]/tmp/ipykernel_178335/867371186.py:7: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  affine_params_true = torch.tensor(affine_params_true)
Train: 100%|██████████| 200/200 [00:32<00:00,  6.15it/s]


In [13]:
test_bar = tqdm(test_dataset, total=len(test_dataset), desc='Test')
for i, data in enumerate(test_bar):

    # Get images and affine parameters
    if model_params.sup:
        source_image, target_image, affine_params_true = data
        affine_params_true = torch.tensor(affine_params_true)
    else:
        source_image, target_image = data
        affine_params_true = None
    source_image = source_image.to(device)
    target_image = target_image.to(device)

    points1, desc1, heatmap1 = superpoint(source_image[0, 0, :, :].cpu().numpy())
    points2, desc2, heatmap2 = superpoint(target_image[0, 0, :, :].cpu().numpy())

    tracker = PointTracker(5, nn_thresh=0.7)
    try:
        matches = tracker.nn_match_two_way(desc1, desc2, nn_thresh=nn_thresh)
    except:
        print('No matches found')
        # TODO: find a better way to do this
        pass

    matches1 = np.array(points1[:2, matches[0, :].astype(int)])
    matches2 = np.array(points2[:2, matches[1, :].astype(int)])
    matches1_2 = transform_points_DVF(torch.tensor(matches1), 
                        affine_params_true, target_image)
    
    # fig, ax = plt.subplots(1, 2)
    # ax[0].imshow(source_image[0, 0, :, :].cpu().detach().numpy())
    # ax[0].plot(matches1[0, :], matches1[1, :], 'r.')
    # ax[1].imshow(target_image[0, 0, :, :].cpu().detach().numpy())
    # ax[1].plot(matches2[0, :], matches2[1, :], 'g.')
    # ax[1].plot(matches1_2[0, :], matches1_2[1, :], 'r.')
    # plt.show()

    # create a dataframe with the matches
    df = pd.DataFrame({'x1': matches1[0, :], 'y1': matches1[1, :],
                       'x2': matches2[0, :], 'y2': matches2[1, :],
                       'x2_': matches1_2[0, :], 'y2_': matches1_2[1, :]})
    save_name = test_df['keypoints'].iloc[i]
    df.to_csv(save_name, index=False)

Test:   0%|          | 0/100 [00:00<?, ?it/s]/tmp/ipykernel_178335/806415840.py:7: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  affine_params_true = torch.tensor(affine_params_true)
Test: 100%|██████████| 100/100 [00:16<00:00,  6.24it/s]


# verify that the saved keypoints are correct